# 04 - Estimation globale du paramètre xi

Dans ce notebook, nous allons construire un estimateur global pour le paramètre xi des GEV aux stations. Nous allons pour cela devoir construire une fonction à $2N+1$ paramètres, où $N$ est le nombre de stations : en effet, il faut estimer en même temps les paramètres $\mu_{i}$, $\phi_{i}$ et enfin $\xi$.

D'après notre précédente étude sur les corrélations des stations d'une même cellule, nous en sommes venus à la conclusion qu'il fallait mieux les concaténer, pour une même cellule. C'est donc ce que nous allons faire.

In [1]:
using CSV, DataFrames, StatsBase, Random

using Distributions, Extremes, Optim

using IDF

import Plots

In [2]:
PROVINCES = ["NB", "NL", "NS", "ON", "PE", "QC"]#provinces considerees

DURATION = "24 h"

"24 h"

In [3]:
station_list = load_data("station_list")

filter!(row -> row.Province ∈ PROVINCES , station_list)#on ne selectionne que les stations qui nous interessent

first(station_list, 10)

,Name,Province,ID,Lat,Lon,Elevation
,String,String,String,Float64,Float64,Int64
1,BEECHWOOD,NB,8100512,46.53,-67.67,91
2,BELLEDUNE,NB,8100514,47.9,-65.83,7
3,BOUCTOUCHE CDA CS,NB,8100593,46.43,-64.77,35
4,CHARLO AUTO,NB,8100885,47.98,-66.33,42
5,MIRAMICHI RCS,NB,8100989,47.02,-65.47,33
6,EDMUNDSTON,NB,8101303,47.42,-68.32,154
7,FREDERICTON A,NB,8101500,45.87,-66.53,20
8,FREDERICTON CDA CS,NB,8101605,45.92,-66.62,35
9,MONCTON INTL A,NB,8103201,46.12,-64.68,70


In [4]:
first(load_station("8100512"), 10)#simple test

,Year,Duration,Pcp
,Int64,Cat…,Float64
1,1959,5 min,9.7
2,1960,5 min,5.3
3,1961,5 min,7.4
4,1962,5 min,5.6
5,1963,5 min,3.6
6,1964,5 min,3.6
7,1965,5 min,11.7
8,1966,5 min,3.0
9,1967,5 min,7.9


Il s'agit maintenant de calculer la log-vraisemblance à estimer, qui devra donc moduler $2N+1$ paramètres. En notant respectivement $\mu_{i}$ et $\phi_{i}$ les paramètres de moyenne et de variance, nous avons donc, avec $(y_{ij})_{1\leq j \leq n_{i}}$ les données de la $i$-ème station :

$$ l(y, \theta) = \sum_{i=1}^{N}\sum_{j=1}^{n_{i}} \ln(GEV(y_{ij}|\mu_{i}, \phi_{i}, \xi)) = \sum_{i=1}^{N} l(y_{i},(\mu_{i}, \phi_{i}, \xi)) $$

In [5]:
function f(p::Vector{:Float64})
    N = div(p, 2)
    
    if p%2 == 0
        error("vector of parameters must be odd long")
    end
    
    μ = p[1:N]
    ϕ = p[N+1:(2*N)]
    ξ = p[2*N+1]
    
    s=0
    for i in 1:N
        id = station_list[i, :ID]
        s -= logL(pcp(id, DURATION), μ[i], ϕ[i], ξ) # on met l'opposé de la log-vraisemblance
    end
    
    return s
end

f (generic function with 1 method)

Avant de tester brutalement avec toutes les stations d'un coup, on pourrait tester par cellules ce que cela donne. Commencons par les extraire :

In [6]:
same_cell = load_data("same_cell")

first(same_cell, 10)

,Name,Province,ID,Lat,Lon,Elevation,GridCell
,String,String,String,Float64,Float64,Int64,Int64
1,ROYAL ROAD,NB,8104480,46.05,-66.72,115,33632
2,ROYAL ROAD WEST,NB,8104482,46.08,-66.73,160,33632
3,PORT WELLER (AUT),ON,6136699,43.25,-79.22,79,20197
4,ST CATHARINES A,ON,6137287,43.2,-79.17,97,20197
5,CAMBRIDGE GALT MOE,ON,6141095,43.33,-80.32,268,18645
6,PRESTON WPCP,ON,6146714,43.38,-80.35,272,18645
7,WATERLOO WELLINGTON A,ON,6149387,43.45,-80.38,317,18645
8,MAPLE,ON,6154950,43.87,-79.48,244,19620
9,TORONTO YORK MILLS,ON,615HHDF,43.75,-79.38,153,19620


In [7]:
cells = unique(same_cell[:, :GridCell])

14-element Vector{Int64}:
 33632
 20197
 18645
 19620
 19619
 19813
 19814
 19621
 28003
 28002
 25464
 28572
 26047
 27434

In [8]:
function xi_gridcell(GridCell::Int64)
    stations = filter(row -> row[:GridCell] == GridCell, same_cell)
    N = nrow(stations)
    
    # initialisation des parametres pour l'optimisation
    μ = []
    ϕ = []
    
    for i in 1:N
        y = pcp(stations[i, :ID], DURATION)
        append!(μ, mean(y))
        append!(ϕ, log(std(y)))
    end
    
    ξ = 0
    
    p₀ = [μ;ϕ;ξ]
    p = p₀
    
    #try 
        res = optimize(f, p₀, LBFGS())
        
        if Optim.converged(res)
            p = Optim.minimizer(res)
        else
            @warn "The maximum likelihood algorithm did not find a solution. Maybe try with different initial values or with another method. The returned values are the initial values."
            p = p₀
        end
    #catch
     #   println("Error with this vector")
    #end
    
    return p
end

xi_gridcell (generic function with 1 method)

In [9]:
for i in cells
    xi_gridcell(i)
end

LoadError: MethodError: no method matching zero(::Type{Any})
[0mClosest candidates are:
[0m  zero(::Type{Union{Missing, T}}) where T at missing.jl:105
[0m  zero([91m::Union{Type{P}, P}[39m) where P<:Dates.Period at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/Dates/src/periods.jl:53
[0m  zero([91m::ForwardDiff.Partials[39m) at /Users/leo/.julia/packages/ForwardDiff/QOqCN/src/partials.jl:39
[0m  ...